In [1]:
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.optim as optim

from models import *
from data_processor import text_processor

In [2]:
T = text_processor()
T.prepare_data("IMDB Dataset.csv")

100%|██████████| 50000/50000 [07:17<00:00, 114.40it/s]


In [12]:
dataset_size = len(T.review_ids)
train_size = int(dataset_size*0.8)
dev_size = dataset_size - train_size

training_reviews = torch.LongTensor(T.review_ids[:train_size])
training_labels = torch.LongTensor(T.labels[:train_size])
training_masks = torch.FloatTensor(T.masks[:train_size])

train_data_set = TensorDataset(training_reviews, training_labels, training_masks)
train_data_loader = DataLoader(train_data_set, batch_size=256, shuffle=True)

dev_reviews = torch.LongTensor(T.review_ids[train_size:])
dev_labels = torch.LongTensor(T.labels[train_size:])
dev_masks = torch.FloatTensor(T.masks[train_size:])

dev_data_set = TensorDataset(dev_reviews, dev_labels, dev_masks)
dev_data_loader = DataLoader(dev_data_set, batch_size=256, shuffle=False)

In [17]:
class Classifier_2(nn.Module):
    def __init__(self, vocab_size, word_dimension):
        super(Classifier_2, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_size, word_dimension)
        self.attention = SelfAttention(word_dimension)
        self.summarisation = Querry_Attention(word_dimension)
        self.fc1 = nn.Linear(word_dimension, 2)

    def forward(self, input_batch, mask):
        embeddings = self.embedding(input_batch)
        sentence_vectors = self.summarisation(embeddings, mask)
        output = self.fc1(sentence_vectors)
        return output


In [21]:
model = Classifier_2(len(T.word_dict),100)

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.8)
loss = nn.CrossEntropyLoss()

In [22]:
for epoch in range(20):
    epoch_loss = 0
    train_accuracy = [0,0]
    i = 0
    for review, labels, mask in train_data_loader:
        y_pred = model(review, mask)
        predictions = torch.argmax(y_pred, dim=1)
        output = loss(y_pred, labels)
        optimizer.zero_grad()
        output.backward()
        optimizer.step()
        epoch_loss += output
        train_accuracy = [train_accuracy[0] + (predictions == labels).sum(), train_accuracy[1] + labels.shape[0]]
        
        #if i%50 == 0:
        #    print(f"{i}: {round(float(output),5)}, {round(int(train_accuracy[0])/int(train_accuracy[1]),5)}")
        #
        #i+=1
        
    dev_epoch_loss = 0
    dev_accuracy = [0,0]

    for review_d, labels_d, mask_d in dev_data_loader:
        y_pred_d = model(review_d, mask_d)
        predictions_d = torch.argmax(y_pred_d, dim=1)
        output_d = loss(y_pred_d, labels_d)
        dev_epoch_loss += output_d
        dev_accuracy = [dev_accuracy[0] + (predictions_d == labels_d).sum(), dev_accuracy[1] + labels_d.shape[0]]
    
    print(f"{epoch}: {round(float(epoch_loss/train_size),5)}, {round(int(train_accuracy[0])/int(train_accuracy[1]),5)}, {round(float(dev_epoch_loss/dev_size),5)}, {round(int(dev_accuracy[0])/int(dev_accuracy[1]),5)}")

0: 0.00275, 0.50414, 0.00271, 0.52853
1: 0.00269, 0.54746, 0.00266, 0.56132
2: 0.00261, 0.57981, 0.00254, 0.59943
3: 0.00254, 0.5968, 0.00251, 0.60891
4: 0.00252, 0.60377, 0.0025, 0.61141
5: 0.00251, 0.60554, 0.00249, 0.6137
6: 0.0025, 0.60935, 0.00249, 0.61446
7: 0.00249, 0.61463, 0.00248, 0.61827
8: 0.00248, 0.62081, 0.00246, 0.6222
9: 0.00246, 0.6277, 0.00245, 0.62677
10: 0.00244, 0.63149, 0.00244, 0.6345
11: 0.00243, 0.63718, 0.00242, 0.63951
12: 0.00242, 0.64006, 0.00242, 0.64082
13: 0.00241, 0.643, 0.00241, 0.64234
14: 0.00241, 0.64556, 0.0024, 0.64648
15: 0.0024, 0.64954, 0.0024, 0.64844
16: 0.0024, 0.65038, 0.00239, 0.64975
17: 0.00239, 0.65202, 0.00239, 0.65269
18: 0.00239, 0.65327, 0.00239, 0.65356
19: 0.00239, 0.65319, 0.00238, 0.65117
